## SNR
The SNR-ratio can be defined by the following function:
\begin{equation}
    SNR = |V_{12}|^2 \cdot \alpha \cdot A \cdot \Phi \cdot \sqrt{\frac{T}{\Delta t}}
\end{equation}
where 
\begin{align}
    |V_{12}| &: \text{complex visiblity [unitless]} \\
    \alpha &: \text{efficiency  [unitless]} \\
    A &: \text{area in [m$^2$]} \\
    \Phi &: \text{spectral photon flux in [m$^{−2}$ s$^{−1}$ Hz$^{−1}$]} \\
    T &: \text{observation or integration time in [s] or [hrs]} \\
    \Delta t &: \text{time resolution in [s]}
\end{align}

In [1]:
import numpy as np
from brightstar_functions import Phi
from scipy.constants import c

In [18]:
V_12 = 1
alpha = 0.125
diameter = 1 # m
effective_diameter = 0.95 # how much of the telescope area actually is used to collect photons
area = np.pi*(diameter*effective_diameter/2)**2 #m^2
area = 0.2
print(area)
delta_t = 3e-11 # s
wavelength = 540e-9 # m, bandwidth of V-mag

def SNR_simple(T, magnitude):
    return V_12**2*alpha*area*Phi(magnitude, wavelength)*np.sqrt(T/delta_t)

mag = 2

#calculate SNR
print(Phi(mag, wavelength))
print('SNR: ', SNR_simple(3600, mag))

0.2
7.821445649871708e-06
SNR:  2.1419911073676943


More explicitly we can express SNR as:
\begin{equation}
    SNR = |V_{12}|^2 \cdot \varepsilon_o \cdot \varepsilon_d \cdot \frac{A}{1+B/\Phi} \cdot \Phi \cdot \sqrt{\frac{T}{\Delta t}} \cdot \sqrt{N_\text{chan}}
\end{equation}
where we additionally have
\begin{align}
    \epsilon_o &: \text{optical efficiency [unitless]} \\
    \epsilon_d &: \text{detector or quantum efficiency [unitless]} \\
    B &: \text{background flux in [m$^{−2}$ s$^{−1}$ Hz$^{−1}$]} \\
    N_{chan} &: \text{number of channels [unitless]} \\
\end{align}

In [19]:
V_12 = 1
efficiency_o = 0.5
efficiency_d = 0.25
B = 0
N_channel = 1
diameter = 1 # m
effective_diameter = 0.95 # how much of the telescope area actually is used to collect photons
area = np.pi*(diameter*effective_diameter/2)**2 #m^2
delta_t = 3e-11 # s
wavelength = 520e-9 # m, bandwidth of V-mag

def SNR_(T, magnitude):
    efficiency = efficiency_o*efficiency_d
    Phi_ = Phi(magnitude, wavelength)
    area_ = area/((1+B/Phi_))
    return V_12**2*area_*Phi_*np.sqrt(T/delta_t)*efficiency*np.sqrt(N_channel)

mag = 2

#calculate SNR
print(Phi(mag, wavelength))
print('SNR: ', SNR_(3600, mag))

7.531762477654238e-06
SNR:  7.310285586117602


For two telescopes of different diameters, efficiencies, background flux and time resolution we have:
\begin{align}
    SNR &= |V_{12}|^2 \cdot \sqrt{\varepsilon_{o,1}\varepsilon_{o,2}} \cdot \sqrt{\varepsilon_{d,1}\varepsilon_{d,2}} \cdot \sqrt{\frac{A_1A_2}{(1+B_1/\Phi)(1+B_2/\Phi)}}\cdot \Phi \cdot
    \sqrt{\frac{T}{\sqrt{\Delta t_1\Delta t_2}}} \cdot \sqrt{N_\text{chan}}
\end{align}

In [20]:
V_12 = 1
epsilon_O_1 = 0.5
epsilon_O_2 = 0.5
epsilon_D_1 = 0.25
epsilon_D_2 = 0.25

B_1 = 0
B_2 = 0
N_channels = 1 


delta_t_1 = 3e-11
delta_t_2 = 3e-11

diameter_1 = 1 # m
diameter_2 = 1 # m
effective_diameter_1 = 0.95 # how much of the telescope area actually is used to collect photons
effective_diameter_2 = 0.95 # how much of the telescope area actually is used to collect photons

area_1 = np.pi*(diameter_1*effective_diameter_1/2)**2 #m^2
area_2 = np.pi*(diameter_2*effective_diameter_2/2)**2 #m^2


wavelength = 520e-9 # m, bandwidth of V-mag

def SNR_two(T, magnitude):
    efficiency = np.sqrt(epsilon_O_1*epsilon_O_1)*np.sqrt(epsilon_D_1*epsilon_D_1)
    Phi_ = Phi(magnitude, wavelength)
    area_ = np.sqrt(area_1*area_2/((1+B_1/Phi_)*(1+B_2/Phi_)))
    return V_12**2*area*Phi_*np.sqrt(T/np.sqrt(delta_t_1*delta_t_2))*efficiency*np.sqrt(N_channel)

mag = 2

#calculate SNR
print('SNR: ', SNR_two(3600, mag))

SNR:  7.310285586117602


We can also check the source count per second:
\begin{equation}
    N_\text{count} &= \varepsilon_o \cdot \epsilon_d \cdot \Phi \cdot \Delta \nu \cdot A\\
\end{equation}
with
\begin{align}
    \Delta \nu = \frac{c\cdot \Delta \lambda}{\lambda^2} \\
\end{align}

In [ ]:
efficiency_d = 0.25
efficiency_o = 0.5
diameter = 1 # m
effective_diameter = 0.95 # how much of the telescope area actually is used to collect photons
area = np.pi*(diameter*effective_diameter/2)**2 #m^2
area = 0.2
delta_t = 3e-11 # s
wavelength = 540e-9 # m, bandwidth of V-mag
delta_lambda = 1e-9 # nm

def count_rate(mag):
    Phi_ = Phi(mag, wavelength)
    delta_nu = c*delta_lambda/(wavelength**2)
    print(delta_nu)
    return efficiency_o*efficiency_d*delta_nu*Phi_*area

print('count rate: ', count_rate(mag))

## Random Coincidence Count
The random coincidence count (in other words, the baseline) per second is:
\begin{equation}
    N_\text{random} &= \varepsilon_o \cdot \epsilon_d \cdot \Phi \cdot \Delta \nu \cdot A\\
\end{equation}

In [25]:
efficiency_d = 0.25
efficiency_o = 0.5
diameter = 1 # m
effective_diameter = 0.95 # how much of the telescope area actually is used to collect photons
area = np.pi*(diameter*effective_diameter/2)**2 #m^2
area = 0.2
delta_t = 30e-12 # s
wavelength = 540e-9 # m, bandwidth of V-mag
delta_lambda = 1e-9 # nm

def coincidences(T, magnitude):
    efficiency = efficiency_d*efficiency_o
    phi = Phi(magnitude, wavelength)
    delta_nu = c*delta_lambda/(wavelength**2)
    return ((area*efficiency*phi*delta_nu)**2)*delta_t*T


mag = 2
print('coincidences in a hour for mag 2: ', coincidences(3600, mag))

coincidences in a hour for mag 2:  4364.59753441479
